In [ ]:
import pandas as pd
import numpy as np
import re

def Energy():
    
    #Import dataset,remove first two columns,skip header and footer,fill missing values,deal with '...' values
    Energy = pd.read_excel(r'Energy Indicators.xls', usecols=[2, 3, 4, 5],
    names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'],
    skiprows = list(range(1, 18)), 
    skipfooter=283-246+1, 
    na_values=['...'])

    # rename country
    
    #remove digits after country name
    Energy['Country'] = Energy.Country.str.replace('[0-9]+?$', '',)
    #remove paranthesis after country name
    Energy['Country'] = Energy.Country.str.replace(' \(.*\)', '',)

    #rename country names
    Energy.replace({"Republic of Korea":"South Korea", 
    "United States of America": "United States", 
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"}, regex = True, inplace=True)
    
    #set the index by country names
    Energy.set_index('Country', inplace=True)

    # convert petajoules to gigajoules
    Energy['Energy Supply'] = Energy['Energy Supply'].apply(lambda x: x*1000000)

    return Energy

def gdp():
    GDP = pd.read_csv('world_bank.csv', skiprows=list(range(0,4)))
    GDP.replace({'Korea, Rep.':'South Korea', 'Iran, Islamic Rep.':'Iran', 'Hong Kong SAR, China':'Hong Kong'},regex = True,inplace=True)
    GDP.set_index('Country Name', inplace=True)
    
    return GDP

def scimen():
    ScimEn = pd.read_excel(r'scimagojr-3.xlsx',index_col = 'Country')
    
    return ScimEn

def answer_one():
    energy=Energy()
    GDP=gdp()
    ScimEn=scimen()
    
    GDP1=GDP.loc[:,'2006':'2015']
    ScimEn1=ScimEn[ScimEn['Rank']<16]
    
    df1=pd.merge(ScimEn1,energy,how='inner',left_index=True,right_index=True)
    df=pd.merge(df1,GDP1,how='inner',left_index=True,right_index=True)
    
    return df

answer_one()

In [ ]:
def answer_two():
    energy=Energy()
    GDP=gdp()
    ScimEn=scimen()
    
    GDP=GDP.loc[:,'2006':'2015']
    
    temp1=pd.merge(ScimEn,energy,how='outer',left_index=True,right_index=True)
    df_outer=pd.merge(temp1,GDP,how='outer',left_index=True,right_index=True)
    
    df=answer_one()
    
    result=df_outer.shape[0]-df.shape[0]
    
    return result

answer_two()

In [ ]:
def answer_three():
    top15=answer_one()
    top15.fillna(0,inplace=True)
    result=top15.loc[:,'2006':'2015'].groupby(top15.index).apply(np.average,axis=1)
    result.sort_values(ascending=False,inplace=True)
    return result

answer_three()

In [ ]:
def answer_four():
    temp=answer_one()
    temp1=answer_three()
    name=temp1.index[5]
    temp2=temp.loc[name]

    result=temp2.loc['2015']-temp2.loc['2006']
    return result

answer_four()

In [ ]:
def answer_five():
    df=answer_one()
    return np.mean(df['Energy Supply per Capita'])

answer_five()

In [ ]:
def answer_six():
    df=answer_one()
    max_renewable=np.max(df['% Renewable'])

    country_name=df[df['% Renewable']==max_renewable].index[0]

    result=(country_name,max_renewable)
    return result

answer_six()

In [ ]:
def answer_seven():
    df=answer_one()
    df.columns
    df['Ratio']=df['Self-citations']/df['Citations']
    max_ratio=np.max(df['Ratio'])
    max_ratio

    country_name=df[df['Ratio']==max_ratio].index[0]
    result=(country_name,max_ratio)
    return result

answer_seven()

In [ ]:
#use df from answer_one()
def answer_eight():
    df=answer_one()
    df['Population Estimate']=df['Energy Supply']/df['Energy Supply per Capita']
    population=df['Population Estimate']
    population=population.sort_values(ascending=False)
    return population.index[2]

answer_eight()

In [ ]:
def answer_nine():
    Top15=answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']

    temp=Top15[['Citable docs per Capita','Energy Supply per Capita']]
    return temp.corr(method='pearson').iloc[0,1]

answer_nine()

In [ ]:
def answer_ten():    
    df=answer_one()
    median_renewable=np.median(df['% Renewable'])
    list1=[]

    for i in range(len(df)):
        if(df['% Renewable'][i]>=median_renewable):
            list1.append(1)
        else:
            list1.append(0)

    df['% Renewabe Measure']=list1
    df.sort_values('Rank',inplace=True)

    HighRenew=df['% Renewabe Measure']
    return HighRenew

answer_ten()

In [ ]:
def answer_eleven():
    df=answer_one()
    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}

    continents=pd.Series(ContinentDict)
    df['Continents']=continents
    df['PopEstimate']=df['Energy Supply']/df['Energy Supply per Capita']
    df.head()

    size=[]

    groups=df.groupby('Continents')
    groups=list(groups)
    #groups[1][1].shape[0]
    for i in range(len(groups)):
        size.append(groups[i][1].shape[0])

    df1=df.groupby('Continents')['PopEstimate'].agg({'sum':np.sum,'mean':np.mean,'std':np.std})

    df1.insert(0,'size',size)

    return df1

answer_eleven()

In [ ]:
def answer_thirteen():
    df=answer_one()
    df['PopEstimate']=df['Energy Supply']/df['Energy Supply per Capita']

    def place_value(number): 
        return ("{:,}".format(number))

    list1=[]

    #place_value(df['PopEstimate'][0])

    for i in range(len(df)):
        list1.append(place_value(df['PopEstimate'][i]))

    PopEst=pd.Series(list1,index=df.index)
    return PopEst

answer_thirteen()